[View in Colaboratory](https://colab.research.google.com/github/sleepless-se/python_reference/blob/master/RNN_alice_pub_002.ipynb)

## 概要

このプログラムはアリス・イン・ワンダーランド（英語版）をディープラーニングで学習させ、その学習データを元に文章の生成を行います。

前回のこちらのファイルをアップデートしたものです。　https://colab.research.google.com/drive/18hpMf8-lAPr9WCedHZmRXfe1h1TuAZRb

### 使い方

1. このファイルは閲覧専用です。「ファイル → ドライブにコピーを保存」で実行できるようになります。

2. 書籍：アリス・イン・ワンダーランド（UTF-8）をここからダウンロードします。

 http://www.gutenberg.org/files/11/11-0.txt

 保存時のファイル名は「alice.txt」にしてください。


3. [3] でサーバーにアップロードします。

※ プログラムは各行「Shift＋Enter」で実行できます。

（上から連打していけばあとは勝手に実行されます。）


## ポイント

### 前回からの変更点


1. ディープラーニングの学習モデルを SimpleRNN から LSTM に変更。
1. Dropoutの追加


####LSTMのメリット
* 学習精度がSimpleRNNより高い

####LSTMのデメリット
* SimpleRNNより学習に時間がかかる

###Dropoutの効果
Dropoutを追加することで学習データにランダムにノイズを加えた状態で学習を行います。そうすることでより予測の制度が向上します。

あとは、過学習を防ぐのにも有効です。


## 重要

実行にすごく時間がかかるので、メニューの「編集」→「ノートブックの設定」から「ハードウェアアクセラレータ」を「GPU」に変更してください。

学習の様子を眺めていると楽しいのですが時間を失うので、他のことをして戻ってきたら「終わってた」というのがベストです。目安は40分くらいです。

In [1]:
from keras.layers import Dense , Activation,LSTM,Dropout
from keras.models import Sequential 
import numpy as np


Using TensorFlow backend.


In [0]:
from google.colab import files

In [3]:
file = files.upload()
!ls

Saving alice.txt to alice.txt
alice.txt  datalab  sample_data


In [4]:
FILE_PATH = "./alice.txt"
text=""
with open(FILE_PATH, 'r',encoding="utf-8") as f:
  for line in f:
    lines = line.split()
    text += " ".join(lines)
text = text.lower()
len(text)

158782

In [0]:
chars = set(text)
nb_chars = len(chars)
char2index = dict((c,i) for i , c in enumerate(chars))
index2char = dict((i,c) for i , c in enumerate(chars))

In [0]:
SEQLEN = 10
STEP = 1

input_chars = []
label_chars = []

for i in range( 0,len(text) - SEQLEN , STEP ):
  input_chars.append(text[i:i+SEQLEN])  
  label_chars.append(text[i+SEQLEN])  
  
  

In [0]:
X = np.zeros((len(input_chars),SEQLEN,nb_chars),dtype=np.bool)
Y = np.zeros((len(input_chars),nb_chars),dtype=np.bool)
for i , input_char in enumerate(input_chars):
  for j , ch in enumerate(input_char):
    X[i,j,char2index[ch]] = 1
  Y[i,char2index[label_chars[i]]] = 1
    


次の行を実行すると学習と学習過程で生成される文章を確認できます。

文章が少しずつきれいになっていく様子をお楽しみください。

※ Generating form sead の次の行が生成された文章です。

In [0]:
HIDDEN_SIZE = 128 
BATCH_SIZE = 128 
NUM_ITERATIONS = 50 
NUM_EPOCHS_PER_ITERATION = 1 
NUM_PREDS_PER_EPOCH = 100 

model = Sequential() 
model.add(LSTM( HIDDEN_SIZE , return_sequences = False , input_shape = ( SEQLEN , nb_chars ),unroll = True )) 
model.add(Dropout(0.2)) 
model.add(Dense(nb_chars)) 
model.add(Activation("softmax")) 
model.compile( loss = "categorical_crossentropy" , optimizer = "rmsprop" )

for iteration in range(NUM_ITERATIONS):
  print("=" * 50)
  print("Iteration #: %d" % (iteration))
  history = model.fit(X,Y,batch_size=BATCH_SIZE,epochs=NUM_EPOCHS_PER_ITERATION)
  test_index = np.random.randint(len(input_chars))
  test_chars = input_chars[test_index]
  print("Generating form sead: {}".format(test_chars))
  print(test_chars , end="")
  for i in range(NUM_PREDS_PER_EPOCH):
    Xtest = np.zeros((1,SEQLEN,nb_chars))
    for i , ch in enumerate(test_chars):
      Xtest[0,i,char2index[ch]]=1
    pred = model.predict(Xtest,verbose=0)[0]
    ypred = index2char[np.argmax(pred)]
    print(ypred,end="")
    test_chars = test_chars[1:] + ypred
  print() 
   


こちらのサイトからタイトルを選択し、

http://www.gutenberg.org/browse/scores/top

「Plain Text UTF-8」をクリックすると今回使用した「アリス・イン・ワンダーランド」と同様にファイルをダウンロードできます。

そのファイルを［３］でアップロードをするとそのタイトル風の文章の生成ができます。

いろいろためして、遊んでみてくださいね(^^)
